In [41]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib 
import gzip

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [24]:
Modelo1 = pd.read_csv('./EDA/Data-EDA/steam_games_EDA.csv.gz',compression='gzip')

In [25]:
Modelo1

,genres,app_name,specs,price,id,developer,año
0,Strategy,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
1,Action,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
2,Indie,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
3,Casual,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
4,Simulation,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
...,...,...,...,...,...,...,...
784882,VR,Maze run vr,HTC Vive,4.99,681550.0,Otros,2016
784883,VR,Maze run vr,Oculus Rift,4.99,681550.0,Otros,2016
784884,VR,Maze run vr,Tracked Motion Controllers,4.99,681550.0,Otros,2016
784885,VR,Maze run vr,Standing,4.99,681550.0,Otros,2016


In [26]:
#Limpieza y preprocesamiento de texto
Modelo1['combined_features'] = Modelo1['genres'] + ' ' + Modelo1['developer'] + ' ' + Modelo1['specs']

In [27]:
Modelo1.shape

(784887, 8)

In [28]:
Modelo1 = Modelo1.groupby('id').first().reset_index()

In [29]:
Modelo1 = Modelo1[Modelo1['id']!=0]

In [30]:
Modelo1['app_name'].nunique()

31285

In [31]:
Modelo1 = Modelo1.sample(n=3500)

In [32]:
Modelo1.shape

(3500, 8)

In [33]:
Modelo1.reset_index(drop=True, inplace=True)

In [34]:
Modelo1

,id,genres,app_name,specs,price,developer,año,combined_features
0,388475.0,Simulation,Euro truck simulator 2 - wheel tuning pack,Single-player,2.99,Scs software,2016,Simulation Scs software Single-player
1,429470.0,Adventure,Space pilgrim episode i: alpha centauri,Single-player,1.99,Grabthegames,2015,Adventure Grabthegames Single-player
2,753002.0,Strategy,Vega conflict - hellfire battleship pack,Multi-player,69.99,Kixeye,2017,Strategy Kixeye Multi-player
3,358230.0,Action,Paganitzu,Single-player,4.99,3d realms (apogee software),1991,Action 3d realms (apogee software) Single-player
4,580833.0,Indie,Rwby: grimm eclipse - team jnpr beacon dance c...,Single-player,1.99,Rooster teeth games,2017,Indie Rooster teeth games Single-player
...,...,...,...,...,...,...,...,...
3495,534440.0,FPS,Killing zombies,Single-player,4.99,Otros,2016,FPS Otros Single-player
3496,553950.0,Action,Gridd: retroenhanced,Single-player,11.99,Kongregate,2017,Action Kongregate Single-player
3497,333810.0,Action,Scroll set,Single-player,0.99,Acquire corp.,2015,Action Acquire corp. Single-player
3498,503340.0,Casual,Dig 4 destruction,Single-player,11.99,Otros,2016,Casual Otros Single-player


In [36]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(Modelo1['combined_features'])

In [37]:
cosine_similarity = cosine_similarity(converted_metrix)

In [38]:
def get_recommendations(app_name, cosine_sim=cosine_similarity ):
    idx = Modelo1[Modelo1['app_name'] == app_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 10 juegos similares
    game_indices = [i[0] for i in sim_scores]
    return Modelo1['app_name'].iloc[game_indices]

In [39]:
get_recommendations('Paganitzu')

963                    Secret agent
1680                      Dark ages
3355               Arctic adventure
1175     Blake stone: planet strike
2545    Blake stone: aliens of gold
Name: app_name, dtype: object

In [42]:
# Se guarda la matriz
joblib.dump(cosine_similarity, 'Modelo1.pkl', compress=1)

['Modelo1.pkl']

In [44]:
Modelo1.to_csv('./Modelo1.csv',index=False)

In [45]:
with gzip.open('./Modelo1.csv.gz', 'wb') as f:
    Modelo1.to_csv(f, index=False, encoding='utf-8')

In [46]:
with open('Modelo1.pkl', 'rb') as file:
        modelo = joblib.load(file)



Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [48]:
reviews2 = pd.read_csv('./Data/Data-Limpia/user_reviews_limpio.csv')

In [49]:
steam2 = pd.read_csv('./EDA/Data-EDA/steam_games_EDA.csv')

In [50]:
Modelo2 = pd.merge(reviews2, steam2, left_on='item_id', right_on='id', how='inner')

In [51]:
Modelo2

,user_id,item_id,recommend,sentiment_analysis,genres,app_name,specs,price,id,developer,año
0,76561197970982479,1250.0,True,2,FPS,Killing floor,Single-player,19.99,1250.0,Tripwire interactive,2009
1,76561197970982479,1250.0,True,2,FPS,Killing floor,Multi-player,19.99,1250.0,Tripwire interactive,2009
2,76561197970982479,1250.0,True,2,FPS,Killing floor,Co-op,19.99,1250.0,Tripwire interactive,2009
3,76561197970982479,1250.0,True,2,FPS,Killing floor,Cross-Platform Multiplayer,19.99,1250.0,Tripwire interactive,2009
4,76561197970982479,1250.0,True,2,FPS,Killing floor,Steam Achievements,19.99,1250.0,Tripwire interactive,2009
...,...,...,...,...,...,...,...,...,...,...,...
7295607,vinquility,431510.0,True,2,Anime,Mystic destinies: serendipity of aeons,In-App Purchases,0.00,431510.0,Aeon dream studios,2016
7295608,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Single-player,0.00,431510.0,Aeon dream studios,2016
7295609,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Steam Achievements,0.00,431510.0,Aeon dream studios,2016
7295610,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Steam Trading Cards,0.00,431510.0,Aeon dream studios,2016
